# **Proyecto Final Inteligencia Artificial**

### Autores: **Angel David Piñeros Sierra**, **Camilo Andrés Roncancio Toca**, **Kelly Johana Solano Calderón**
### Presentado a: **Darwin Eduardo Martinez Riaño**


---

## **Modelo de segmentación de imágenes para la localización de lesiones asociadas al cáncer de piel**

### *Glosario*

### *(A) Descripción de la problemática*

### *(B) Objetivo*

### *(C) Descripción del dataset*

El dataset seleccionado para la evaluación del modelo fue el denominado “Skin cancer: HAM10000” de la plataforma de Kaggle, el cual ofrece un conjunto de imágenes especiales para realizar tareas de segmentación y clasificación. Para el propósito de segmentación, el dataset incluye para cada una de las imágenes, el conjunto de máscaras qué determinan la segmentación de las lesiones de cáncer de piel. 

> El acrónimo HAM10000 significa “Human Against Machine with 10000 training images”. 

Este dataset es una recopilación de imágenes demoscópicas de diferentes poblaciones. Estas fueron originalmente publicadas inicialmente en el repositorio de Harvard Dataverse,  con el propósito de abordar la dificultad de encontrar un dataset lo suficientemente grande y diverso para realizar diagnósticos automatizados de lesiones cutáneas pigmentadas. 

El dataset se conforma de dos carpetas: images y masks. Cada una con **10015** imágenes en formato **.JPEG**. Todas las imágenes tienen una dimensión de `600px X 450px`

<img src="https://res.cloudinary.com/dlsntlruu/image/upload/v1764556079/carpeta_images_pieoyu.png" width="600px"/>

<img src="https://res.cloudinary.com/dlsntlruu/image/upload/v1764556079/carpeta_masks_taifwn.png" width="600px"/>

Las imágenes incluyen diagnósticos de:
*  Queratosis actínicas
*  Carcinoma intraepitelial
*  Carcinoma basocelular
*  Lesiones de tipo queratosis
*  Dermatofibroma
*  Melanoma
*  Lesiones vasculares

Contar con una amplia gama de diagnósticos permite qué la tarea de segmentación semántica pueda realizarse de forma óptima. 

Para mayor información: 

*  Skin cancer: HAM10000: https://www.kaggle.com/datasets/surajghuwalewala/ham1000-segmentation-and-classification/
*  The HAM10000 dataset, a large collection of multi-source dermatoscopic images of common pigmented skin lesions: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T



### *(D) Importación y organización de datos*


In [48]:
%pip install kaggle pandas torch PILlow torchvision matplotlib

  Using cached matplotlib-3.10.7-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached contourpy-1.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.7-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (8.7 MB)
Using cached contourpy-1.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (362 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 7.2 MB/s  0:00:00 eta 0:00:01
Using cached kiwisolver-1.4.9-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (1.5 MB)
Using cached pyparsing-3.2.5-py3-none-any.whl (113 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [matplo

In [49]:
import os
import zipfile
import pandas as pd
import subprocess
import random
import shutil
from pathlib import Path
import torch
from PIL import Image
from torchvision import transforms
import torchvision.transforms.v2 as v2
from torchvision import tv_tensors
from torchvision.transforms import InterpolationMode
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path

In [34]:


dataset_url = "surajghuwalewala/ham1000-segmentation-and-classification"
ruta_descarga = "ham1000_data"

if os.path.exists(ruta_descarga):
    print(f"La carpeta '{ruta_descarga}' ya existe. No se descargará de nuevo.")
else:
    os.makedirs(ruta_descarga, exist_ok=True)
    subprocess.run([
        "kaggle", "datasets", "download",
        "-d", dataset_url,
        "-p", ruta_descarga,
        "--unzip"
    ], check=True)

print("Contenido de la carpeta:", os.listdir(ruta_descarga))

images_dir = os.path.join(ruta_descarga, "images")
masks_dir = os.path.join(ruta_descarga, "masks")

if os.path.isdir(images_dir):
    print("Número de imágenes:", len(os.listdir(images_dir)))
else:
    print("No encontré la carpeta 'images'.")

if os.path.isdir(masks_dir):
    print("Número de máscaras:", len(os.listdir(masks_dir)))
else:
    print("No encontré la carpeta 'masks'.")

La carpeta 'ham1000_data' ya existe. No se descargará de nuevo.
Contenido de la carpeta: ['GroundTruth.csv', 'images', 'masks']
Número de imágenes: 10017
Número de máscaras: 10015


In [35]:
DATA_ROOT = Path('ham1000_data')
OUTPUT_ROOT = Path('ham1000_data_splits')
TRAIN_RATIO = 0.85  # train + val
VAL_FROM_TRAIN = 0.15  # porcentaje de train destinado a validación
SEED = 42
MASK_SUFFIX = '_segmentation.png'

image_dir = DATA_ROOT / 'images'
mask_dir = DATA_ROOT / 'masks'
if not image_dir.exists() or not mask_dir.exists():
    raise RuntimeError(f"No se hallaron carpetas esperadas en {DATA_ROOT}.")

random.seed(SEED)
if OUTPUT_ROOT.exists():
    shutil.rmtree(OUTPUT_ROOT)
for split in ('train', 'val', 'test'):
    (OUTPUT_ROOT / split / 'images').mkdir(parents=True, exist_ok=True)
    (OUTPUT_ROOT / split / 'masks').mkdir(parents=True, exist_ok=True)

allowed_exts = {'.png', '.jpg', '.jpeg'}
image_paths = [p for p in image_dir.iterdir() if p.suffix.lower() in allowed_exts]
if not image_paths:
    raise RuntimeError(f'No se encontraron imágenes en {image_dir}.')
image_paths.sort()
random.shuffle(image_paths)

total = len(image_paths)
train_val_count = int(total * TRAIN_RATIO)
val_count = int(train_val_count * VAL_FROM_TRAIN)
train_count = train_val_count - val_count

splits = {
    'train': image_paths[:train_count],
    'val': image_paths[train_count: train_count + val_count],
    'test': image_paths[train_count + val_count:]
}

for split_name, files in splits.items():
    dst_img = OUTPUT_ROOT / split_name / 'images'
    dst_mask = OUTPUT_ROOT / split_name / 'masks'
    for img_path in files:
        mask_path = mask_dir / f"{img_path.stem}{MASK_SUFFIX}"
        if not mask_path.exists():
            raise FileNotFoundError(f'Falta máscara: {mask_path}')
        shutil.copy2(img_path, dst_img / img_path.name)
        shutil.copy2(mask_path, dst_mask / mask_path.name)
    print(f"{split_name}: {len(files)} imágenes")
print('Splits creados en', OUTPUT_ROOT)



train: 7236 imágenes
val: 1276 imágenes
test: 1503 imágenes
Splits creados en ham1000_data_splits


In [36]:
TRAIN_IMG_DIR = Path('ham1000_data_splits/train/images')
if not TRAIN_IMG_DIR.exists():
    raise RuntimeError(f'No existe el directorio: {TRAIN_IMG_DIR}')

TARGET_SIZE = (256, 256)
resize_transform = transforms.Resize(TARGET_SIZE, interpolation=InterpolationMode.BILINEAR)

img_files = sorted(
    [p for p in TRAIN_IMG_DIR.iterdir() if p.suffix.lower() in {'.png', '.jpg', '.jpeg'}]
)
if not img_files:
    raise RuntimeError(f'No se encontraron imágenes en {TRAIN_IMG_DIR}')

means, stds = [], []
for img_path in img_files:
    img = Image.open(img_path).convert('RGB')
    img_tensor = transforms.ToTensor()(resize_transform(img))
    means.append(img_tensor.mean(dim=(1, 2)))
    stds.append(img_tensor.std(dim=(1, 2)))

IMG_MEAN = torch.stack(means).mean(dim=0)
IMG_STD = torch.stack(stds).mean(dim=0)
print(f'Media por canal (reescala {TARGET_SIZE[0]}x{TARGET_SIZE[1]}): {IMG_MEAN.tolist()}')
print(f'Desviación estándar por canal: {IMG_STD.tolist()}')



Media por canal (reescala 256x256): [0.7633206844329834, 0.5454925894737244, 0.5698098540306091]
Desviación estándar por canal: [0.08873128145933151, 0.11731848120689392, 0.13180507719516754]


In [50]:
normalize_transform = v2.Normalize(mean=IMG_MEAN, std=IMG_STD)

train_joint_transform = v2.Compose([
    v2.Resize(TARGET_SIZE, interpolation=InterpolationMode.BILINEAR, antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.2),
    v2.RandomRotation(degrees=(-180, 180)),
    v2.ElasticTransform(alpha=40.0, sigma=5.0, interpolation=InterpolationMode.BILINEAR),
    v2.ColorJitter(brightness=0.15, contrast=0.2, saturation=0.2, hue=0.02),
    v2.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0)),
    v2.ToDtype(torch.float32, scale=True),
    normalize_transform,
])

val_test_joint_transform = v2.Compose([
    v2.Resize(TARGET_SIZE, interpolation=InterpolationMode.BILINEAR, antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    normalize_transform,
])

def apply_train_transforms(image, mask):
    image_tv = tv_tensors.Image(image)
    mask_tv = tv_tensors.Mask(mask)
    image_aug, mask_aug = train_joint_transform(image_tv, mask_tv)
    return image_aug, mask_aug

def apply_val_transforms(image, mask):
    image_tv = tv_tensors.Image(image)
    mask_tv = tv_tensors.Mask(mask)
    image_val, mask_val = val_test_joint_transform(image_tv, mask_tv)
    return image_val, mask_val

def apply_test_transforms(image, mask):
    image_tv = tv_tensors.Image(image)
    mask_tv = tv_tensors.Mask(mask)
    image_test, mask_test = val_test_joint_transform(image_tv, mask_tv)
    return image_test, mask_test

print("Transformaciones definidas:")
print("train_joint_transform:", train_joint_transform)
print("val_test_joint_transform:", val_test_joint_transform)



Transformaciones definidas:
train_joint_transform: Compose(
      Resize(size=[256, 256], interpolation=InterpolationMode.BILINEAR, antialias=True)
      RandomHorizontalFlip(p=0.5)
      RandomVerticalFlip(p=0.2)
      RandomRotation(degrees=[-180.0, 180.0], interpolation=InterpolationMode.NEAREST, expand=False, fill=0)
      ElasticTransform(alpha=[40.0, 40.0], sigma=[5.0, 5.0], interpolation=InterpolationMode.BILINEAR, fill=0)
      ColorJitter(brightness=(0.85, 1.15), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.02, 0.02))
      GaussianBlur(kernel_size=(3, 3), sigma=[0.1, 1.0])
      ToDtype(scale=True)
      Normalize(mean=[tensor(0.7633), tensor(0.5455), tensor(0.5698)], std=[tensor(0.0887), tensor(0.1173), tensor(0.1318)], inplace=False)
)
val_test_joint_transform: Compose(
      Resize(size=[256, 256], interpolation=InterpolationMode.BILINEAR, antialias=True)
      ToDtype(scale=True)
      Normalize(mean=[tensor(0.7633), tensor(0.5455), tensor(0.5698)], std=[tensor(0.0

In [51]:
from torch.utils.data import Dataset

class HAM1000SegmentationDataset(Dataset):
    def __init__(self, root_dir: Path, split: str, transform_fn=None, mask_suffix: str = '_segmentation.png'):
        self.root_dir = Path(root_dir)
        if split not in {'train', 'val', 'test'}:
            raise ValueError("split debe ser 'train', 'val' o 'test'")
        self.split = split
        self.images_dir = self.root_dir / split / 'images'
        self.masks_dir = self.root_dir / split / 'masks'
        if not self.images_dir.exists() or not self.masks_dir.exists():
            raise RuntimeError(f'No se hallan carpetas para el split {split} en {self.root_dir}')
        self.mask_suffix = mask_suffix
        self.transform_fn = transform_fn

        allowed_exts = {'.png', '.jpg', '.jpeg'}
        self.samples = []
        for img_path in sorted(self.images_dir.iterdir()):
            if img_path.suffix.lower() not in allowed_exts:
                continue
            mask_path = self.masks_dir / f"{img_path.stem}{self.mask_suffix}"
            if not mask_path.exists():
                continue
            self.samples.append((img_path, mask_path))
        if not self.samples:
            raise RuntimeError(f'No se encontraron pares imagen-máscara en {self.images_dir}')

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, mask_path = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')
        if self.transform_fn is not None:
            image, mask = self.transform_fn(image, mask)
        return image, mask


def get_transform_fn(split: str):
    if split == 'train':
        return apply_train_transforms
    if split == 'val':
        return apply_val_transforms
    if split == 'test':
        return apply_test_transforms
    raise ValueError('split desconocido')


def create_datasets(root_dir: Path = Path('ham1000_data_splits')):
    datasets = {}
    for split in ('train', 'val', 'test'):
        datasets[split] = HAM1000SegmentationDataset(
            root_dir=root_dir,
            split=split,
            transform_fn=get_transform_fn(split),
        )
        print(f"{split}: {len(datasets[split])} muestras")
    return datasets

ham_datasets = create_datasets()



train: 7236 muestras
val: 1276 muestras
test: 1503 muestras


In [52]:
BATCH_SIZE = 8
NUM_WORKERS = 4
PIN_MEMORY = True

train_loader = DataLoader(
    ham_datasets['train'],
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
)
val_loader = DataLoader(
    ham_datasets['val'],
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
)
test_loader = DataLoader(
    ham_datasets['test'],
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
)

print('Loaders listos:')
print(f"train: {len(train_loader)} batches")
print(f"val: {len(val_loader)} batches")
print(f"test: {len(test_loader)} batches")



Loaders listos:
train: 905 batches
val: 160 batches
test: 188 batches


In [53]:
class ResidualBlock(nn.Module):
    """Two Conv-BN blocks with a residual shortcut."""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        if in_channels != out_channels:
            self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        identity = self.shortcut(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = out + identity
        return F.relu(out)

class Down(nn.Module):
    """MaxPool + ResidualBlock encoder stage."""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.resblock = ResidualBlock(in_channels, out_channels)

    def forward(self, x):
        x = self.pool(x)
        x = self.resblock(x)
        return x

class Up(nn.Module):
    """ConvTranspose2d upsampling + concat + ResidualBlock."""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.resblock = ResidualBlock(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diff_y = x2.size(2) - x1.size(2)
        diff_x = x2.size(3) - x1.size(3)
        if diff_y != 0 or diff_x != 0:
            x1 = F.pad(x1, [diff_x // 2, diff_x - diff_x // 2, diff_y // 2, diff_y - diff_y // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.resblock(x)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class ResUNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=2):
        super().__init__()
        self.inc = ResidualBlock(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        self.up1 = Up(1024, 512)
        self.up2 = Up(512, 256)
        self.up3 = Up(256, 128)
        self.up4 = Up(128, 64)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        return self.outc(x)

def get_device():
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [54]:
def _ensure_multiclass_logits(preds):
    if preds.ndim != 4:
        raise ValueError('preds debe tener forma [B, C, H, W]')
    if preds.shape[1] == 1:
        probs = torch.sigmoid(preds)
        return torch.cat([1.0 - probs, probs], dim=1)
    return preds

def pixel_accuracy(preds, targets):
    preds = _ensure_multiclass_logits(preds)
    pred_labels = torch.argmax(preds, dim=1)
    correct = (pred_labels == targets).float().sum()
    total = torch.numel(targets)
    return (correct / total).item()

def _per_class_iou(pred_labels, targets, num_classes=2, eps=1e-6):
    per_class = {}
    for cls in range(num_classes):
        pred_cls = pred_labels == cls
        target_cls = targets == cls
        intersection = (pred_cls & target_cls).float().sum()
        union = (pred_cls | target_cls).float().sum()
        if union == 0:
            continue
        per_class[cls] = ((intersection + eps) / (union + eps)).item()
    return per_class

def _per_class_dice(pred_labels, targets, num_classes=2, eps=1e-6):
    per_class = {}
    for cls in range(num_classes):
        pred_cls = pred_labels == cls
        target_cls = targets == cls
        intersection = (pred_cls & target_cls).float().sum()
        pred_sum = pred_cls.float().sum()
        target_sum = target_cls.float().sum()
        denom = pred_sum + target_sum
        if denom == 0:
            continue
        per_class[cls] = ((2 * intersection + eps) / (denom + eps)).item()
    return per_class

def compute_jaccard(preds, targets, num_classes=2, eps=1e-6):
    preds = _ensure_multiclass_logits(preds)
    pred_labels = torch.argmax(preds, dim=1)
    per_class = _per_class_iou(pred_labels, targets, num_classes, eps)
    if not per_class:
        return {'mean': 0.0, 'per_class': per_class}
    mean_score = sum(per_class.values()) / len(per_class)
    return {'mean': mean_score, 'per_class': per_class}

def compute_dice(preds, targets, num_classes=2, eps=1e-6):
    preds = _ensure_multiclass_logits(preds)
    pred_labels = torch.argmax(preds, dim=1)
    per_class = _per_class_dice(pred_labels, targets, num_classes, eps)
    if not per_class:
        return {'mean': 0.0, 'per_class': per_class}
    mean_score = sum(per_class.values()) / len(per_class)
    return {'mean': mean_score, 'per_class': per_class}


In [55]:
from pathlib import Path

WEIGHTS_PATH = Path('resunet_gsd_lr5e4.pt')
device = get_device()
model = ResUNet().to(device)

checkpoint = torch.load(WEIGHTS_PATH, map_location=device)
state_dict = (
    checkpoint.get('state_dict')
    or checkpoint.get('model_state_dict')
    or checkpoint
)
for key in ('outc.conv.weight', 'outc.conv.bias'):
    state_dict.pop(key, None)
missing = model.load_state_dict(state_dict, strict=False)
model.eval()
print(f'Modelo cargado desde {WEIGHTS_PATH}')
print('Dispositivo:', device)
print('Keys ignoradas:', missing.missing_keys)


Modelo cargado desde resunet_gsd_lr5e4.pt
Dispositivo: cpu
Keys ignoradas: ['outc.conv.weight', 'outc.conv.bias']


In [56]:
import math
import random

random.seed(45)

OPTIMIZERS = ('adam', 'adamw', 'sgd')
BATCH_CHOICES = (8, 12, 16)
WEIGHT_DECAY_VALUES = (0.0, 1e-4, 5e-4)
MOMENTUM_VALUES = (0.85, 0.9, 0.95, 0.99)
SCHEDULERS = {
    'none': {},
    'steplr': {'step_size': (15, 30), 'gamma': (0.5, 0.1)},
    'cosine': {'t_max': (20, 40)},
    'plateau': {'patience': (3, 5), 'factor': (0.3, 0.1)},
}

def sample_lr(opt_name):
    if opt_name == 'sgd':
        return 10 ** random.uniform(-2.3, -1.5)
    if opt_name == 'adamw':
        return 10 ** random.uniform(-4.3, -3.0)
    return 10 ** random.uniform(-4.0, -3.0)

def sample_scheduler_params(name):
    params = {}
    cfg = SCHEDULERS[name]
    for key, bounds in cfg.items():
        params[key] = random.choice(bounds)
    return params

def sample_trial():
    opt_name = random.choice(OPTIMIZERS)
    batch_size = random.choice(BATCH_CHOICES)
    weight_decay = random.choice(WEIGHT_DECAY_VALUES)
    lr = sample_lr(opt_name) * (batch_size / 8)
    scheduler = random.choice(tuple(SCHEDULERS.keys()))
    trial = {
        'optimizer': opt_name,
        'learning_rate': round(lr, 6),
        'batch_size': batch_size,
        'weight_decay': weight_decay,
        'scheduler': scheduler,
        'scheduler_params': sample_scheduler_params(scheduler),
    }
    if opt_name == 'sgd':
        trial['momentum'] = random.choice(MOMENTUM_VALUES)
    return trial

NUM_TRIALS = 30
random_trials = [sample_trial() for _ in range(NUM_TRIALS)]

for idx, cfg in enumerate(random_trials, start=1):
    print(f'Trial {idx}:')
    for key, value in cfg.items():
        print(f'  {key}: {value}')
    print('-' * 30)

# Itera random_trials para lanzar entrenamientos y guardar métricas


Trial 1:
  optimizer: adamw
  learning_rate: 0.000163
  batch_size: 12
  weight_decay: 0.0001
  scheduler: cosine
  scheduler_params: {'t_max': 40}
------------------------------
Trial 2:
  optimizer: adam
  learning_rate: 0.000103
  batch_size: 8
  weight_decay: 0.0001
  scheduler: cosine
  scheduler_params: {'t_max': 20}
------------------------------
Trial 3:
  optimizer: adamw
  learning_rate: 0.000112
  batch_size: 16
  weight_decay: 0.0005
  scheduler: none
  scheduler_params: {}
------------------------------
Trial 4:
  optimizer: adam
  learning_rate: 0.000214
  batch_size: 12
  weight_decay: 0.0005
  scheduler: plateau
  scheduler_params: {'patience': 3, 'factor': 0.1}
------------------------------
Trial 5:
  optimizer: adam
  learning_rate: 0.001675
  batch_size: 16
  weight_decay: 0.0
  scheduler: steplr
  scheduler_params: {'step_size': 30, 'gamma': 0.1}
------------------------------
Trial 6:
  optimizer: adamw
  learning_rate: 0.000133
  batch_size: 16
  weight_decay: 0.

In [57]:
import time
from pathlib import Path
from copy import deepcopy
import torch
import torch.nn as nn

NUM_CLASSES = 2
EPS = 1e-6

class MetricAccumulator:
    def __init__(self, num_classes=NUM_CLASSES, eps=EPS):
        self.num_classes = num_classes
        self.eps = eps
        self.reset()

    def reset(self):
        self.intersection = torch.zeros(self.num_classes, dtype=torch.float64)
        self.union = torch.zeros(self.num_classes, dtype=torch.float64)
        self.pred_sum = torch.zeros(self.num_classes, dtype=torch.float64)
        self.target_sum = torch.zeros(self.num_classes, dtype=torch.float64)
        self.correct = 0.0
        self.total = 0.0

    def update(self, logits, targets):
        preds = _ensure_multiclass_logits(logits.detach())
        pred_labels = torch.argmax(preds, dim=1)
        targets = targets.long()
        self.correct += (pred_labels == targets).sum().item()
        self.total += targets.numel()
        for cls in range(self.num_classes):
            pred_mask = pred_labels == cls
            target_mask = targets == cls
            intersection = (pred_mask & target_mask).sum().item()
            union = (pred_mask | target_mask).sum().item()
            self.intersection[cls] += intersection
            self.union[cls] += union
            self.pred_sum[cls] += pred_mask.sum().item()
            self.target_sum[cls] += target_mask.sum().item()

    def export(self):
        per_class_iou = {}
        per_class_dice = {}
        for cls in range(self.num_classes):
            if self.union[cls] > 0:
                per_class_iou[cls] = (self.intersection[cls] + self.eps) / (self.union[cls] + self.eps)
            if (self.pred_sum[cls] + self.target_sum[cls]) > 0:
                per_class_dice[cls] = (2 * self.intersection[cls] + self.eps) / (self.pred_sum[cls] + self.target_sum[cls] + self.eps)
        mean_iou = sum(per_class_iou.values()) / len(per_class_iou) if per_class_iou else 0.0
        mean_dice = sum(per_class_dice.values()) / len(per_class_dice) if per_class_dice else 0.0
        acc = self.correct / self.total if self.total else 0.0
        return {
            'pixel_acc': acc,
            'jaccard': {'mean': mean_iou, 'per_class': per_class_iou},
            'dice': {'mean': mean_dice, 'per_class': per_class_dice},
        }

def prepare_targets(mask_tensor):
    if mask_tensor.ndim == 4 and mask_tensor.size(1) == 1:
        mask_tensor = mask_tensor.squeeze(1)
    mask_tensor = (mask_tensor > 0).long()
    return mask_tensor

def build_optimizer(model, cfg):
    params = [p for p in model.parameters() if p.requires_grad]
    if cfg['optimizer'] == 'adam':
        return torch.optim.Adam(params, lr=cfg['learning_rate'], weight_decay=cfg['weight_decay'])
    if cfg['optimizer'] == 'adamw':
        return torch.optim.AdamW(params, lr=cfg['learning_rate'], weight_decay=cfg['weight_decay'])
    return torch.optim.SGD(
        params,
        lr=cfg['learning_rate'],
        momentum=cfg.get('momentum', 0.9),
        weight_decay=cfg['weight_decay'],
    )

def build_scheduler(optimizer, cfg, total_epochs):
    name = cfg.get('scheduler', 'none')
    params = cfg.get('scheduler_params', {}) or {}
    if name == 'steplr':
        return torch.optim.lr_scheduler.StepLR(
            optimizer,
            step_size=params.get('step_size', 30),
            gamma=params.get('gamma', 0.1),
        )
    if name == 'cosine':
        return torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=params.get('t_max', total_epochs),
        )
    if name == 'plateau':
        return torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='max',
            patience=params.get('patience', 4),
            factor=params.get('factor', 0.3),
        )
    return None

def prepare_base_state(weights_path, device):
    if weights_path is None:
        return None
    weights_path = Path(weights_path)
    if not weights_path.exists():
        print(f"Advertencia: no se encontró {weights_path}")
        return None
    checkpoint = torch.load(weights_path, map_location=device)
    state_dict = (
        checkpoint.get('state_dict')
        or checkpoint.get('model_state_dict')
        or checkpoint
    )
    state_dict = dict(state_dict)
    state_dict.pop('outc.conv.weight', None)
    state_dict.pop('outc.conv.bias', None)
    return state_dict

def summarize_metrics(split_name, metrics):
    dice = metrics['dice']
    jacc = metrics['jaccard']
    parts = [
        f"{split_name} loss={metrics['loss']:.4f}",
        f"pixAcc={metrics['pixel_acc']:.4f}",
        f"dice={dice['mean']:.4f}",
        f"jacc={jacc['mean']:.4f}",
    ]
    for cls, score in sorted(dice['per_class'].items()):
        parts.append(f"dice_c{cls}={score:.4f}")
    for cls, score in sorted(jacc['per_class'].items()):
        parts.append(f"jacc_c{cls}={score:.4f}")
    return ' | '.join(parts)

def train_or_eval_epoch(
    model,
    loader,
    optimizer,
    criterion,
    device,
    train_mode=True,
    loss_type='cross_entropy',
    num_classes=NUM_CLASSES,
):
    model.train(mode=train_mode)
    accumulator = MetricAccumulator(num_classes=num_classes)
    running_loss = 0.0
    data_count = len(loader.dataset)
    with torch.set_grad_enabled(train_mode):
        for images, masks in loader:
            images = images.to(device)
            targets = prepare_targets(masks.to(device))
            logits = model(images)
            if loss_type == 'bce':
                if logits.ndim == 3:
                    logits = logits.unsqueeze(1)
                target_float = targets.unsqueeze(1).float()
                loss = criterion(logits, target_float)
            else:
                loss = criterion(logits, targets.long())
            if train_mode:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            running_loss += loss.item() * images.size(0)
            accumulator.update(logits, targets)
    metrics = accumulator.export()
    metrics['loss'] = running_loss / max(1, data_count)
    return metrics

def run_single_trial(
    cfg,
    num_epochs=10,
    loss_type='cross_entropy',
    base_state_dict=None,
    weights_path=None,
):
    device = get_device()
    output_channels = NUM_CLASSES if loss_type != 'bce' else 1
    model = ResUNet(n_channels=3, n_classes=output_channels).to(device)
    loaded_state = None
    if base_state_dict is not None:
        loaded_state = base_state_dict
    elif weights_path is not None:
        loaded_state = prepare_base_state(weights_path, device)
    if loaded_state is not None:
        model.load_state_dict(loaded_state, strict=False)
    criterion = nn.BCEWithLogitsLoss() if loss_type == 'bce' else nn.CrossEntropyLoss()
    optimizer = build_optimizer(model, cfg)
    scheduler = build_scheduler(optimizer, cfg, total_epochs=num_epochs)
    history = []
    best_val = -float('inf')
    best_state = None
    for epoch in range(1, num_epochs + 1):
        start = time.time()
        train_metrics = train_or_eval_epoch(
            model,
            train_loader,
            optimizer,
            criterion,
            device,
            train_mode=True,
            loss_type=loss_type,
            num_classes=NUM_CLASSES,
        )
        val_metrics = train_or_eval_epoch(
            model,
            val_loader,
            optimizer,
            criterion,
            device,
            train_mode=False,
            loss_type=loss_type,
            num_classes=NUM_CLASSES,
        )
        monitor = val_metrics['dice']['mean']
        if scheduler:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(monitor)
            else:
                scheduler.step()
        if monitor > best_val:
            best_val = monitor
            best_state = deepcopy(model.state_dict())
        epoch_time = time.time() - start
        history.append(
            {
                'epoch': epoch,
                'train': train_metrics,
                'val': val_metrics,
                'lr': optimizer.param_groups[0]['lr'],
                'time': epoch_time,
            }
        )
        train_summary = summarize_metrics('train', train_metrics)
        val_summary = summarize_metrics('val', val_metrics)
        print(
            f"[Trial {cfg['optimizer']} | Epoch {epoch}] {train_summary} || {val_summary} || "
            f"lr={optimizer.param_groups[0]['lr']:.2e}"
        )
    return {
        'config': cfg,
        'history': history,
        'best_val_dice': best_val,
        'best_state_dict': best_state,
    }

def run_trials(
    trials,
    num_epochs=10,
    loss_type='cross_entropy',
    weights_path=None,
    top_k=3,
    output_dir=Path('experiments'),
):
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    device = get_device()
    base_state_dict = prepare_base_state(weights_path, device) if weights_path else None
    results = []
    top_models = []
    for idx, cfg in enumerate(trials, start=1):
        print(f'===== Trial {idx}/{len(trials)} =====')
        result = run_single_trial(
            cfg,
            num_epochs=num_epochs,
            loss_type=loss_type,
            base_state_dict=base_state_dict,
        )
        results.append(result)
        dice_score = result['best_val_dice']
        save_path = output_dir / f"trial_{idx:03d}_dice_{dice_score:.4f}.pt"
        torch.save(
            {
                'config': result['config'],
                'state_dict': result['best_state_dict'],
                'history': result['history'],
                'best_val_dice': dice_score,
            },
            save_path,
        )
        top_models.append({'score': dice_score, 'path': save_path, 'config': result['config']})
        top_models.sort(key=lambda item: item['score'], reverse=True)
        while len(top_models) > top_k:
            removed = top_models.pop()
            if removed['path'].exists():
                removed['path'].unlink()
        print('Top actual:')
        for rank, item in enumerate(top_models, start=1):
            print(f"  {rank}) Dice={item['score']:.4f} -> {item['path'].name}")
    return {'results': results, 'top_models': top_models}

In [58]:
import torch
import matplotlib.pyplot as plt

def _extract_metric(history, split, metric, cls=None):
    values = []
    for record in history:
        entry = record[split]
        if metric == 'loss':
            values.append(entry['loss'])
        elif metric == 'pixel_acc':
            values.append(entry['pixel_acc'])
        elif metric in {'dice', 'jaccard'}:
            if cls is None:
                values.append(entry[metric]['mean'])
            else:
                values.append(entry[metric]['per_class'].get(cls, float('nan')))
        else:
            raise ValueError(f'Métrica desconocida: {metric}')
    return values

def plot_top_models(summary, metrics=('loss', 'pixel_acc', 'dice', 'jaccard'), splits=('train', 'val')):
    if not summary or 'top_models' not in summary:
        raise ValueError('Debes proporcionar el resumen retornado por run_trials')
    top_models = summary['top_models']
    if not top_models:
        raise ValueError('No hay modelos en el ranking top-k')
    for rank, entry in enumerate(top_models, start=1):
        payload = torch.load(entry['path'])
        history = payload['history']
        epochs = [rec['epoch'] for rec in history]
        fig, axes = plt.subplots(len(metrics), 1, figsize=(10, 4 * len(metrics)), sharex=True)
        if len(metrics) == 1:
            axes = [axes]
        fig.suptitle(f"Top {rank} | Dice={entry['score']:.4f} | {entry['path'].name}")
        for ax, metric in zip(axes, metrics):
            for split in splits:
                if metric in {'dice', 'jaccard'}:
                    mean_values = _extract_metric(history, split, metric)
                    ax.plot(epochs, mean_values, label=f"{split}-{metric}-mean")
                    for cls in range(NUM_CLASSES):
                        cls_values = _extract_metric(history, split, metric, cls=cls)
                        ax.plot(epochs, cls_values, linestyle='--', alpha=0.6, label=f"{split}-{metric}-cls{cls}")
                else:
                    values = _extract_metric(history, split, metric)
                    ax.plot(epochs, values, label=f"{split}-{metric}")
            ax.set_ylabel(metric)
            ax.grid(True)
            ax.legend(loc='best')
        axes[-1].set_xlabel('Epoch')
        plt.tight_layout()
        plt.show()

summary = run_trials(random_trials, num_epochs=10, loss_type='cross_entropy', weights_path=WEIGHTS_PATH, top_k=3)
plot_top_models(summary)


===== Trial 1/30 =====


/home/angel/Bureau/Proyecto IA/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 